In [1]:
import pandas as pd
import numpy as np
from jinja2 import Environment, FileSystemLoader

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##### POG594, NORMAL: P01596, TUMOUR: P01871, TUMOUR RNA: P01874
##### POG664, NORMAL:P01821, TUMOUR: P01829, TUMOUR RNA: P01833

In [6]:
cols = ['chr', 'pos',  'orig_real', 'orig_comment', 'Real (Simon)', 'Comment (Simon)',
       'Real (Erin)', 'Comment (Erin)', 'P01596_Alt_Count', 'P01596_Ref_Count',
       'P01871_Alt_Count', 'P01871_Ref_Count', 'P01874_Alt_Count',
       'P01874_Ref_Count', 'VAF']
f = '/projects/trans_scratch/validations/workspace/szong/development/deepreview/POG594_all.somatic.coding.snvs.merged.xlsx'
df = pd.read_excel(f, sheet_name='POG594_all', usecols=cols)
df.shape
df.head()

(329, 15)

,chr,pos,orig_real,orig_comment,Real (Simon),Comment (Simon),Real (Erin),Comment (Erin),P01596_Alt_Count,P01596_Ref_Count,P01871_Alt_Count,P01871_Ref_Count,P01874_Alt_Count,P01874_Ref_Count,VAF
0,1,1222164,yes,automatic pass,Y,0,NaN,NaN,0,34,8,37,9,46,0.177778
1,1,17607217,yes,automatic pass,Y,0,yes,0,0,26,24,13,71,0,0.648649
2,1,39879379,maybe,1 good G and 3 low quality Gs in blood.,N,Messy alignments in both tumour and normal. N...,no,0,2,19,6,20,0,5,0.230769
3,1,44489918,yes,automatic pass,Y,0,NaN,NaN,0,23,21,29,16,1,0.420000
4,1,67288099,yes,automatic pass,Y,0,yes,0,0,19,15,47,2,3,0.241935


In [59]:
cols = ['chr', 'pos', 'orig_real', 'orig_comment', 'Real (Yaoqing)', 'Comment (Yaoqing)',
       'Real (Erin)', 'Comment (Erin)', 'P01821_Alt_Count', 'P01821_Ref_Count',
       'P01829_Alt_Count', 'P01829_Ref_Count', 'P01833_Alt_Count',
       'P01833_Ref_Count', 'VAF']
f = '/projects/trans_scratch/validations/workspace/szong/development/deepreview/POG664_all.somatic.coding.snvs.merged.xlsx'
df = pd.read_excel(f, sheet_name='POG664_all', usecols=cols)
df.shape
df.head()

(146, 15)

,chr,pos,orig_real,orig_comment,Real (Yaoqing),Comment (Yaoqing),Real (Erin),Comment (Erin),P01821_Alt_Count,P01821_Ref_Count,P01829_Alt_Count,P01829_Ref_Count,P01833_Alt_Count,P01833_Ref_Count,VAF
0,1,8384631,fail,NaN,autofail,prev failed + no strelka + ms<0.85,NaN,NaN,1,31,5,78,0,5,0.060241
1,1,26608897,fail,NaN,autofail,prev failed + no strelka + ms<0.85,NaN,NaN,0,21,3,61,1,195,0.046875
2,1,39879250,maybe,1 good G and 3 low quality Gs in blood.,no,"0 map quality reads, messy alignment, same pat...",no,0,3,15,8,16,0,12,0.333333
3,1,39879643,yes,NaN,no,"multiple variants nearby, looks like alignment...",NaN,NaN,1,27,8,91,1,32,0.080808
4,1,65858293,yes,automatic pass,yes,0,NaN,NaN,0,31,13,70,6,14,0.156627


In [62]:
# df['Real (Simon)'].unique()
df['Real (Yaoqing)'].unique()

array(['autofail', 'no', 'yes', 'maybe'], dtype=object)

In [33]:
df_real = df[df['Real (Simon)']== 'Y']
df_real.shape

df[df['Real (Simon)']== 'Maybe'].shape

df_false = df[(df['Real (Simon)']== 'N')|(df['Real (Simon)']== 'autofail')]
df_false.shape

(185, 17)

In [63]:
df_real = df[df['Real (Yaoqing)']== 'yes']
df_real.shape

df[df['Real (Yaoqing)']== 'maybe'].shape

df_false = df[(df['Real (Yaoqing)']== 'no')|(df['Real (Yaoqing)']== 'autofail')]
df_false.shape

(45, 15)

(13, 15)

(88, 15)

In [64]:
rl = 150

In [65]:
def get_positions(sub, n, rs, offset):
#     sub = df.sample(n=n, weights='target', random_state=rs).reset_index(drop=True)
#     sub = df
    sub.sort_values(['chr', 'pos'], inplace=True)
    if offset:
        print('offset positions!')
        nrows = sub.shape[0]
        # random choose position that is 2 to 50 base away from a real variant as negative
        a = [1 if np.random.random() > 0.5 else -1 for i in range(nrows)]
        b = np.random.randint(2, 100, nrows)
        sub['pos'] += a * b
        sub = sub.drop(['ref', 'alt', 'genotype', 'target'], axis=1)
    sub['start'] = sub.pos - rl
    sub['end'] = sub.pos + rl
    sub = sub.astype(str)
    sub['positions'] = sub['chr'] + ':' + sub['start'] + '-' + sub['end']
    return sub

In [66]:
dfa = get_positions(df, df.shape[0], 1, 0)

In [67]:
dfa.shape
dfa.head(2)
all_positions = dfa.positions

(146, 18)

,chr,pos,orig_real,orig_comment,Real (Yaoqing),Comment (Yaoqing),Real (Erin),Comment (Erin),P01821_Alt_Count,P01821_Ref_Count,P01829_Alt_Count,P01829_Ref_Count,P01833_Alt_Count,P01833_Ref_Count,VAF,start,end,positions
0,1,8384631,fail,nan,autofail,prev failed + no strelka + ms<0.85,nan,nan,1,31,5,78,0,5,0.060240963855421686,8384481,8384781,1:8384481-8384781
1,1,26608897,fail,nan,autofail,prev failed + no strelka + ms<0.85,nan,nan,0,21,3,61,1,195,0.046875,26608747,26609047,1:26608747-26609047


In [68]:
dfsub = get_positions(df_false, df_false.shape[0], 111, 0)

/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [75]:
dfsub = get_positions(df_real, df_real.shape[0], 111, 0)

/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/projects/da_workspace/software/miniconda/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [76]:
dfsub.head()
dfsub.shape

,chr,pos,orig_real,orig_comment,Real (Yaoqing),Comment (Yaoqing),Real (Erin),Comment (Erin),P01821_Alt_Count,P01821_Ref_Count,P01829_Alt_Count,P01829_Ref_Count,P01833_Alt_Count,P01833_Ref_Count,VAF,start,end,positions
4,1,65858293,yes,automatic pass,yes,0,nan,nan,0,31,13,70,6,14,0.1566265060240964,65858143,65858443,1:65858143-65858443
9,1,152636832,yes,automatic pass,yes,0,nan,nan,0,33,37,57,na,na,0.39361702127659576,152636682,152636982,1:152636682-152636982
12,1,157509124,yes,automatic pass,yes,0,nan,nan,0,31,13,75,0,18,0.14772727272727273,157508974,157509274,1:157508974-157509274
13,1,158225111,yes,automatic pass,yes,0,yes?,low vaf but clean,0,30,12,83,0,57,0.12631578947368421,158224961,158225261,1:158224961-158225261
14,1,158596774,yes,automatic pass,yes,0,yes?,low vaf but clean,0,39,11,68,na,na,0.13924050632911392,158596624,158596924,1:158596624-158596924


(45, 18)

a few notes:
* it is tricky to get the right igv screen shot. i need to manually adjust the igv window to roughly 900x700 pixels.run igv -b command on one snapshot image and manually adjust the igv window size to 900x700. Actually, the height can not be controlled. I can only control the width, the height will reset to the previous maximum height when you taking screenshot. So it does not matter. it depends on the coverage. It seems I need to have a option to downsample. otherwise, it only shows the first 30x or so reads.
* it is not a viable option to attempt to keep the position of interest in the middle. It is hard to preprocessing. Also cnn has the ability to detect the image at any position. we use the centre line as a cue. this way i can do data augmentation as well. presumably, the model will generalize better.
* I need to be consistent when generate those screenshots, remember to use collapse mode, zoom to the last 5th level.igv window size is 739. 

* many parameter/options in igv batch file does not really work. for example: maxtrackheight, sort tec.

* the height pixels for the screen shot changes depending on the read coverage, 

* i need to be very careful when cropping the images. cropping need to start from right and top. i may need to preprocess the images before feed to fastai imagelist.

* remember some of those randomly generated negative images may be empty due to zero coverage, remove these before feeding to model

### generate 2000 real variant igv screen shots

In [77]:
positions = dfsub.positions

In [78]:
def render_template(template_dir, out_file, template_file, positions):
    jinja2_env = Environment(loader=FileSystemLoader([template_dir]),
                             trim_blocks=True)
    template = jinja2_env.get_template(template_file)
    with open(out_file, 'w') as opf:
        content = template.render(out_file=out_file,
                                  positions=positions)                                 
        opf.write(content)
#         print('templated {}'.format(sh_script))
    return out_file

In [79]:
tmpdir = '//projects/da_workspace/szspace/data/pog664_posorg/templates/'
tmpfile = 'batch_file_template.txt'

In [80]:
def batch_file(tmpdir, tmpfile, bfname, positions):
    bfile = f'{tmpdir}{bfname}'
    render_template(tmpdir, bfile, tmpfile, positions)

In [81]:
bfname = 'pos_batch_file2.txt'
batch_file(tmpdir, tmpfile, bfname, positions)

* bedtools can generate random intervals for genome. -n number of intervals, -l length of interval, -g genome
 -seed random seeds

* need to generate better negative images. we would like to include:
        1)those ones that are 2-50 base pair away from the known variant position.
        2) looks like a real variant, but the supporting reads also have multiple mis-matches at other places. These are the most useful training images
        3) random positions absent from variant

##### 1)those ones that are 2-50 base pair away from the known variant position.

In [159]:
dfsub2 = get_positions(df, 2000, 2, 1)

In [160]:
positions = dfsub2.positions

# make sure the pick neg positions does not have a real variant

In [161]:
any(positions.isin(all_positions))

True

In [139]:
positions = positions[~positions.isin(all_positions)]

In [150]:
positions[:2]

301       1:916684-916984
3901    1:2888364-2888664
Name: positions, dtype: object

In [140]:
bfname = 'neg_batch_file.txt'
batch_file(tmpdir, tmpfile, bfname, positions)

##### 3) random positions absent from variant

In [74]:
# !/projects/da_workspace/software/bedtools2/bin/bedtools random -n 2000 -l 2 -g /projects/da_workspace/szspace/data/giab/hg19.genome -seed 71346|cut -f1,2 > random_positions.txt

In [162]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/random_positions.txt'
df2 = pd.read_csv(f, sep='\t', header=None)
df2.head(2)

,0,1
0,5,33654973
1,12,20645578


In [163]:
df2['start'] = df2[1] - rl
df2['end'] = df2[1] + rl
df2 = df2.astype(str)
df2['positions'] = df2[0] + ':' + df2.start + '-' + df2.end 

In [164]:
df2[~df2.positions.isin(all_positions)].shape

(1999, 5)

In [169]:
df2.head()
negposs = df2.positions

,0,1,start,end,positions
0,5,33654973,33654823,33655123,5:33654823-33655123
1,12,20645578,20645428,20645728,12:20645428-20645728
2,14,66189388,66189238,66189538,14:66189238-66189538
3,X,123437045,123436895,123437195,X:123436895-123437195
4,6,49025787,49025637,49025937,6:49025637-49025937


In [85]:
positions[:2]

['1:1239239-1239439', '1:2288647-2288847']

In [166]:
negposs[:2]

['5:33654823-33655123', '12:20645428-20645728']

In [172]:
any(negposs.isin(all_positions))

False

In [171]:
negposs = negposs[~negposs.isin(all_positions)]

In [87]:
# template_dir = '/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/'
# template_file = 'batch_file_template.txt'
# out_file = f'{template_dir}/neg_batch_file.txt'
# render_template(template_dir, out_file, template_file, negposs)

'/projects/trans_scratch/validations/workspace/szong/Cervical/notebook//neg_batch_file.txt'

In [173]:
bfname = 'neg11_batch_file.txt'
batch_file(tmpdir, tmpfile, bfname, negposs)

# generate some real data from cervical project

In [3]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/rainstorm/all_rainstorm_hotspots_reviewed_from_wei_20190403.csv'
df = pd.read_csv(f, sep='\t')
df.head(2)
df.shape

,chr,start_x,end_x,is_real,comments,patients,num_strela_patients,gene,feature_of_interest,num_normal_patients,...,beta_mean_x,beta_mean_y,beta_mut_mean,beta_wt_mean,meth_bg_diff,meth_mutwt_diff,Tumour libs,Tumour bams,Normal libs,Normal bams
0,6,142706206,142706207,yes,NaN,HTMCP-03-06-02058;HTMCP-03-06-02210;HTMCP-03-0...,4,GPR126,active_enhancer,0,...,0.727533,0.765304,0.83250,0.716387,0.025302,0.074965,A37261;A52339;A52340;A68234,/projects/analysis/analysis22/A37261/merge_bwa...,A37189;A54512;A54515;A66442,/projects/analysis/analysis22/A37189/merge_bwa...
1,6,142706209,142706210,yes,NaN,HTMCP-03-06-02058;HTMCP-03-06-02063;HTMCP-03-0...,8,GPR126,active_enhancer,0,...,0.727533,0.765304,0.71875,0.720551,0.025302,0.001252,A37261;A37266;A37275;A54376;A52336;A68244;A682...,/projects/analysis/analysis22/A37261/merge_bwa...,A37189;A37194;A37203;A54481;A54507;A66452;A664...,/projects/analysis/analysis22/A37189/merge_bwa...


(404, 40)

In [8]:
df.columns

Index(['chr', 'start_x', 'end_x', 'is_real', 'comments', 'patients',
       'num_strela_patients', 'gene', 'feature_of_interest',
       'num_normal_patients', 'num_mutationseq_patient', 'Reference_Allele',
       'Tumor_Seq_Allele2', 'start_y', 'end_y', 'chrom', 'x_pos',
       'feature_colors', 'hiv_status', 'hivpos_count', 'hivneg_count', 'hist',
       'Clade', 'A7_count', 'A9_count', 'overlap_centromere', 'in_gatk_pon',
       'hiv_diff', 'clade_diff', 'clade_colors', 'beta_mean_x', 'beta_mean_y',
       'beta_mut_mean', 'beta_wt_mean', 'meth_bg_diff', 'meth_mutwt_diff',
       'Tumour libs', 'Tumour bams', 'Normal libs', 'Normal bams'],
      dtype='object')

In [10]:
df['Tumour bams'][0]

'/projects/analysis/analysis22/A37261/merge_bwa-mem-0.7.6a/125nt/hg19a/A37261_4_lanes_dupsFlagged.bam;/projects/analysis/analysis22/A52339/merge_bwa-mem-0.7.6a/125nt/hg19a/A52339_4_lanes_dupsFlagged.bam;/projects/analysis/analysis22/A52340/merge_bwa-mem-0.7.6a/125nt/hg19a/A52340_4_lanes_dupsFlagged.bam;/projects/analysis/analysis27/A68234/merge18086_bwa-mem-0.7.6a-sb/150nt/hg19a/A68234_2_lanes_dupsFlagged.bam'

In [13]:
dfs = df[['chr', 'start_x', 'is_real', 'Tumour bams']]
dfs.shape
dfs.head(2)

(404, 4)

,chr,start_x,is_real,Tumour bams
0,6,142706206,yes,/projects/analysis/analysis22/A37261/merge_bwa...
1,6,142706209,yes,/projects/analysis/analysis22/A37261/merge_bwa...


keep only first bams because Wei reviewed them

In [46]:
s = df['Tumour bams'].str.split(';', expand=True).stack()
# s
type(s)
# s.index
s = s[s.index.get_level_values(1)<3]
s[1]
idx = s.index.get_level_values(0)
dfn = dfs.loc[idx]
dfn['bam'] = list(s)
dfn.drop('Tumour bams', axis=1, inplace=True)
dfn['start'] = dfn['start_x'] - 100
dfn['end'] = dfn['start_x'] + 100
dfn = dfn.astype(str)
dfn.head(2)

pandas.core.series.Series

0    /projects/analysis/analysis22/A37261/merge_bwa...
1    /projects/analysis/analysis22/A37266/merge_bwa...
2    /projects/analysis/analysis22/A37275/merge_bwa...
dtype: object

,chr,start_x,is_real,bam,start,end
0,6,142706206,yes,/projects/analysis/analysis22/A37261/merge_bwa...,142706106,142706306
0,6,142706206,yes,/projects/analysis/analysis22/A52339/merge_bwa...,142706106,142706306


In [92]:
dfn['position'] = dfn['chr'] + ':' + dfn['start'] + '-' + dfn['end']
dfn['lib'] = dfn.bam.apply(lambda x: x.split('/')[4])
dfn['fname'] = dfn['lib'] + '_' + dfn['chr'] + '_' + dfn['start'] + '_' + dfn['end'] + '.png'
dfn.head(2)

,chr,start_x,is_real,bam,start,end,position,lib,fname
0,6,142706206,yes,/projects/analysis/analysis22/A37261/merge_bwa...,142706106,142706306,6:142706106-142706306,A37261,A37261_6_142706106_142706306.png
0,6,142706206,yes,/projects/analysis/analysis22/A52339/merge_bwa...,142706106,142706306,6:142706106-142706306,A52339,A52339_6_142706106_142706306.png


In [93]:
dff = dfn.groupby(['bam', 'is_real', 'lib'])['position', 'fname'].agg({'position':';'.join, 'fname': ';'.join}).reset_index()

In [94]:
dff.shape
dff.head(2)

(138, 5)

,bam,is_real,lib,position,fname
0,/projects/analysis/analysis21/A52317/merge_bwa...,no,A52317,14:52218727-52218927;3:4498329-4498529;6:16253...,A52317_14_52218727_52218927.png;A52317_3_44983...
1,/projects/analysis/analysis21/A52318/merge_bwa...,no,A52318,7:142148579-142148779;1:5734753-5734953;6:1866...,A52318_7_142148579_142148779.png;A52318_1_5734...


In [95]:
def render_template(template_dir, out_file, template_file, positions, bam, snapdir, snapnames):
    jinja2_env = Environment(loader=FileSystemLoader([template_dir]),
                             trim_blocks=True)

    jinja2_env.filters['zip'] = zip
    template = jinja2_env.get_template(template_file)
    with open(out_file, 'w') as opf:
        content = template.render(out_file=out_file,
                                  bam=bam,
                                  snapdir=snapdir,
                                  snapnames=snapnames,
                                  positions=positions)                                 
        opf.write(content)
#         print('templated {}'.format(sh_script))
    return out_file

In [96]:
dff.head(2)

,bam,is_real,lib,position,fname
0,/projects/analysis/analysis21/A52317/merge_bwa...,no,A52317,14:52218727-52218927;3:4498329-4498529;6:16253...,A52317_14_52218727_52218927.png;A52317_3_44983...
1,/projects/analysis/analysis21/A52318/merge_bwa...,no,A52318,7:142148579-142148779;1:5734753-5734953;6:1866...,A52318_7_142148579_142148779.png;A52318_1_5734...


In [100]:
template_dir = '/projects/trans_scratch/validations/workspace/szong/Cervical/notebook/'
template_file = 'batch_file_template2.txt'
outdir = '/projects/da_workspace/szspace/data/muts'
for ix, row in dff.iterrows():
    bam = row['bam']
    real = row['is_real']
    poss = row['position'].split(';')
    fnames = row['fname'].split(';')
    lib = row['lib']
    print(poss)

    out_file = f'{outdir}/{real}/{lib}_batch_file.txt'
    render_template(template_dir, out_file, template_file, poss, bam, f'{outdir}/{real}', fnames)

['14:52218727-52218927', '3:4498329-4498529', '6:162536489-162536689', '7:135804347-135804547', '7:142484146-142484346', '8:64641708-64641908', '8:79436265-79436465', '11:24623486-24623686', '11:117122055-117122255', '13:93048062-93048262', '13:114022180-114022380', '17:48163570-48163770', '18:1629361-1629561', '23:88461431-88461631']


'/projects/da_workspace/szspace/data/muts/no/A52317_batch_file.txt'

['7:142148579-142148779', '1:5734753-5734953', '6:18666938-18667138', '20:43392715-43392915']


'/projects/da_workspace/szspace/data/muts/no/A52318_batch_file.txt'

['1:153323062-153323262', '11:69312579-69312779', '12:86531989-86532189', '11:74208290-74208490', '1:231257801-231258001', '3:75741870-75742070', '3:195669693-195669893', '6:4895376-4895576', '12:40289998-40290198', '15:47371067-47371267', '17:48163570-48163770', '19:50974823-50975023', '20:14935455-14935655', '20:14935459-14935659']


...

'/projects/da_workspace/szspace/data/muts/no/A78437_batch_file.txt'

['1:231257801-231258001', '12:38256388-38256588']


'/projects/da_workspace/szspace/data/muts/no/A78440_batch_file.txt'

['11:39877869-39878069']


'/projects/da_workspace/szspace/data/muts/yes/A78440_batch_file.txt'

need to concatenate all batch files into a single file to generate the snapshot. otherwise if will just hang if you loop all files

not sure how the resolution of the pixels is determined when taking a screen shot. They are quite different for the test set from cervical. it can be as large as 512x512.